# Web Mining and Applied NLP (44-620)

## Requests, JSON, and NLP

### Student Name: Monsuru Adebisi
[View My GitHub Repo](https://github.com/don4ye/web-mining-project4.git)

Perform the tasks described in the Markdown cells below.  When you have completed the assignment make sure your code cells have all been run (and have output beneath them) and ensure you have committed and pushed ALL of your changes to your assignment repository.

Make sure you have [installed spaCy and its pipeline](https://spacy.io/usage#quickstart) and [spaCyTextBlob](https://spacy.io/universe/project/spacy-textblob)

Every question that requires you to write code will have a code cell underneath it; you may either write your entire solution in that cell or write it in a python file (`.py`), then import and run the appropriate code to answer the question.

This assignment requires that you write additional files (either JSON or pickle files); make sure to submit those files in your repository as well.

## Question 1
The following code accesses the [lyrics.ovh](https://lyricsovh.docs.apiary.io/#reference/0/lyrics-of-a-song/search) public api, searches for the lyrics of a song, and stores it in a dictionary object.  Write the resulting json to a file (either a JSON file or a pickle file; you choose). You will read in the contents of this file for future questions so we do not need to frequently access the API.

In [1]:
import requests
import json

# Step 1: Fetch the lyrics data from the API
response = requests.get('https://api.lyrics.ovh/v1/They Might Be Giants/Birdhouse in your soul')
result = json.loads(response.text)

# Step 2: Write the JSON data to a file
filename = "lyrics_data.json"
with open(filename, "w") as file:
    json.dump(result, file, indent=4)  # Use indent=4 for better readability

print("Lyrics data has been saved to", filename)


Lyrics data has been saved to lyrics_data.json


## Question 2
Read in the contents of your file.  Print the lyrics of the song (not the entire dictionary!) and use spaCyTextBlob to perform sentiment analysis on the lyrics.  Print the polarity score of the sentiment analysis.  Given that the range of the polarity score is `[-1.0,1.0]` which corresponds to how positive or negative the text in question is, do you think the lyrics have a more positive or negative connotaion?  Answer this question in a comment in your code cell.

In [3]:
import json
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

# Register the SpacyTextBlob component
nlp.add_pipe("spacytextblob", last=True)

# Read the lyrics from the JSON file
with open('lyrics_data.json', 'r') as json_file:  # Use your file name
    lyrics_data = json.load(json_file)

# Extract the lyrics from the dictionary
lyrics = lyrics_data.get('lyrics', '')

# Perform sentiment analysis
doc = nlp(lyrics)
polarity_score = doc._.blob.polarity  # Access polarity from the TextBlob object

# Print the lyrics and the polarity score
print("Lyrics:\n", lyrics)
print("\nPolarity Score:", polarity_score)

# Comment on the sentiment
if polarity_score > 0:
    print("The lyrics have a more positive connotation.")
elif polarity_score < 0:
    print("The lyrics have a more negative connotation.")
else:
    print("The lyrics have a neutral connotation.")


Lyrics:
 I'm your only friend 
I'm not your only friend 
But I'm a little glowing friend 
But really I'm not actually your friend 
But I am 


Blue canary in the outlet by the light switch 

Who watches over you 

Make a little birdhouse in your soul 

Not to put too fine a point on it 

Say I'm the only bee in your bonnet 

Make a little birdhouse in your soul 



I have a secret to tell 

From my electrical well 

It's a simple message and I'm leaving out the whistles and bells 

So the room must listen to me 

Filibuster vigilantly 

My name is blue canary one note* spelled l-i-t-e 

My story's infinite 

Like the Longines Symphonette it doesn't rest 



Blue canary in the outlet by the light switch 

Who watches over you 

Make a little birdhouse in your soul 

Not to put too fine a point on it 

Say I'm the only bee in your bonnet 

Make a little birdhouse in your soul 



I'm your only friend 

I'm not your only friend 

But I'm a little glowing friend 

But really I'm not actual

## Question 3
Write a function that takes an artist, song, and filename, accesses the lyrics.ovh api to get the song lyrics, and writes the results to the specified filename.  Test this function by getting the lyrics to any four songs of your choice and storing them in different files.

In [4]:
import requests
import json

def write_song_from_api_to_json(artist, song, filename):
    """
    Fetches lyrics from the lyrics.ovh API and writes them to a JSON file.
    
    Parameters:
    - artist: The name of the artist (string)
    - song: The name of the song (string)
    - filename: The name of the file to save the lyrics (string, with .json extension)
    """
    # Format the API URL
    url = f"https://api.lyrics.ovh/v1/{artist}/{song}"
    
    # Fetch the lyrics from the API
    response = requests.get(url)
    
    if response.status_code == 200:  # Check if the request was successful
        lyrics_data = response.json()
        
        # Write the lyrics to the specified JSON file
        with open(filename, "w") as file:
            json.dump(lyrics_data, file, indent=4)
        
        print(f"Lyrics for '{song}' by '{artist}' have been saved to {filename}")
    else:
        print(f"Failed to fetch lyrics for '{song}' by '{artist}'. Status Code: {response.status_code}")

# Test the function with four songs of your choice
write_song_from_api_to_json("Taylor Swift", "Love Story", "love_story.json")
write_song_from_api_to_json("Ed Sheeran", "Shape of You", "shape_of_you.json")
write_song_from_api_to_json("Adele", "Hello", "hello.json")
write_song_from_api_to_json("Coldplay", "Yellow", "yellow.json")


Lyrics for 'Love Story' by 'Taylor Swift' have been saved to love_story.json
Lyrics for 'Shape of You' by 'Ed Sheeran' have been saved to shape_of_you.json
Lyrics for 'Hello' by 'Adele' have been saved to hello.json
Lyrics for 'Yellow' by 'Coldplay' have been saved to yellow.json


## Question 4
Write a function that takes the name of a file that contains song lyrics, loads the file, performs sentiment analysis, and returns the polarity score.  Use this function to print the polarity scores (with the name of the song) of the three files you created in question 3.  Does the reported polarity match your understanding of the song's lyrics? Why or why not do you think that might be?  Answer the questions in either a comment in the code cell or a markdown cell under the code cell.

In [6]:
import json
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

# Initialize spaCy and add SpacyTextBlob
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("spacytextblob", last=True)

def analyze_lyrics_sentiment(filename):
    """
    Loads song lyrics from a JSON file, performs sentiment analysis,
    and returns the polarity score.
    
    Parameters:
    - filename: The name of the JSON file containing song lyrics (string)
    
    Returns:
    - The polarity score of the song lyrics (float)
    """
    # Load the lyrics from the JSON file
    with open(filename, "r") as file:
        data = json.load(file)
    
    lyrics = data.get("lyrics", "")
    if not lyrics:
        print(f"No lyrics found in {filename}")
        return None

    # Perform sentiment analysis
    doc = nlp(lyrics)
    polarity_score = doc._.blob.polarity  # Correct way to access polarity with spacytextblob
    return polarity_score

# Use the function on the files created in Question 3
songs = [
    {"name": "Love Story", "file": "love_story.json"},
    {"name": "Shape of You", "file": "shape_of_you.json"},
    {"name": "Hello", "file": "hello.json"},
    {"name": "Yellow", "file": "yellow.json"}
]

# Print the polarity scores for each song
for song in songs:
    score = analyze_lyrics_sentiment(song["file"])
    if score is not None:
        print(f"Polarity Score for '{song['name']}': {score}")

# Comment on the sentiment analysis results
# Does the reported polarity match your understanding of the song's lyrics?
# Answer:
# The polarity scores may not always align perfectly with our understanding of the songs' emotions.
# This could be because sentiment analysis tools primarily focus on the words' sentiment
# rather than the overall context or themes. For example, a song might use negative words
# in a metaphorical or artistic way that the sentiment analyzer doesn't interpret correctly.


Polarity Score for 'Love Story': 0.049999999999999996
Polarity Score for 'Shape of You': 0.31929292929292935
Polarity Score for 'Hello': -0.14109195402298852
Polarity Score for 'Yellow': 0.2430555555555556
